In [2]:
import pandas as pd
import numpy as np

### 크롤링한 데이터 전처리

In [29]:
kin = pd.read_csv("/Users/Jenny/Desktop/공모전/유치원 데이터 크롤링.csv")
kin.head()

,kin_type,kin_name,kin_addr
0,사립(사인),가람유치원,<span>서울특별시 강동구 상암로63길 39</span>
1,사립(법인),가명유치원,<span>서울특별시 중구 청파로 447-1</span>
2,사립(사인),강남유정유치원,<span>서울특별시 강남구 광평로34길 17</span>
3,사립(사인),강남유치원,<span>서울특별시 강남구 선릉로129길 31</span>
4,사립(사인),강남유치원,<span>서울특별시 동작구 상도로41가길 13</span>


In [30]:
kin['kin_addr'] = kin['kin_addr'].str[6:-7]
kin.head()

,kin_type,kin_name,kin_addr
0,사립(사인),가람유치원,서울특별시 강동구 상암로63길 39
1,사립(법인),가명유치원,서울특별시 중구 청파로 447-1
2,사립(사인),강남유정유치원,서울특별시 강남구 광평로34길 17
3,사립(사인),강남유치원,서울특별시 강남구 선릉로129길 31
4,사립(사인),강남유치원,서울특별시 동작구 상도로41가길 13


### 도로명 주소로 행정동 추출하기

In [18]:
# 주소만 적힌 데이터 생성

addrs = kin['kin_addr']
addrs

0        서울특별시 강동구 상암로63길 39
1         서울특별시 중구 청파로 447-1
2        서울특별시 강남구 광평로34길 17
3       서울특별시 강남구 선릉로129길 31
4       서울특별시 동작구 상도로41가길 13
               ...          
753    서울특별시 강동구 천중로35길 37-1
754       서울특별시 은평구 은평터널로 65
755        서울특별시 노원구 마들로 111
756      서울특별시 노원구 중계로18길 14
757     서울특별시 성북구 동소문로34길 24
Name: kin_addr, Length: 758, dtype: object

In [7]:
import requests
import json

locations = []
for addr in addrs:
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(addr)
    headers = {
    ## 카카오 API의 REST API키를 아래 예시와 같이 입력해주세요
    ## "Authorization": "KakaoAK REST API키 입력 gogo"}
    "Authorization": "KakaoAK 8cd64c4299a983cac6fd3edef103f134"}
    place = requests.get(url, headers = headers).json()['documents']
    locations.append(place)

In [8]:
city = [] ## 구
town = [] ## 행정동
for i in range(len(locations)):
    
    try:
        city.append(locations[i][0].get('address').get('region_2depth_name'))
        town.append(locations[i][0].get('address').get('region_3depth_h_name'))
        
    except IndexError:
        print(i,'번째 주소 못가져옴', end ='')
        print()
        city.append('없음')
        town.append('없음')
    
    except AttributeError:
        city.append(locations[i][0].get('road_address').get('region_2depth_name'))
        town.append(locations[i][0].get('road_address').get('region_3depth_h_name'))

3 번째 주소 못가져옴
196 번째 주소 못가져옴


In [9]:
city_town = np.array([city,town]).T
df_temp = pd.DataFrame(city_town, columns = ['region_2depth_name','region_3depth_h_name'])
df_temp.head()

,region_2depth_name,region_3depth_h_name
0,강동구,명일2동
1,중구,중림동
2,강남구,수서동
3,없음,없음
4,동작구,상도1동


In [31]:
kin['addr_gu'] = df_temp['region_2depth_name']
kin['addr_dong'] = df_temp['region_3depth_h_name']
kin.head()

,kin_type,kin_name,kin_addr,addr_gu,addr_dong
0,사립(사인),가람유치원,서울특별시 강동구 상암로63길 39,강동구,명일2동
1,사립(법인),가명유치원,서울특별시 중구 청파로 447-1,중구,중림동
2,사립(사인),강남유정유치원,서울특별시 강남구 광평로34길 17,강남구,수서동
3,사립(사인),강남유치원,서울특별시 강남구 선릉로129길 31,없음,없음
4,사립(사인),강남유치원,서울특별시 동작구 상도로41가길 13,동작구,상도1동


In [32]:
# 데이터 타입 변환

kin['addr_gu'] = kin['addr_gu'].astype(str)
kin['addr_dong'] = kin['addr_dong'].astype(str)

In [33]:
kin[kin['addr_dong'] == '없음']

,kin_type,kin_name,kin_addr,addr_gu,addr_dong
3,사립(사인),강남유치원,서울특별시 강남구 선릉로129길 31,없음,없음
196,공립(단설),서울개포유치원,서울특별시 강남구 영동대로3길 45,없음,없음


In [34]:
# 미운영 데이터 제외 (결측치)

kin = kin.drop(3)
kin = kin.drop(196)
kin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 756 entries, 0 to 757
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   kin_type   756 non-null    object
 1   kin_name   756 non-null    object
 2   kin_addr   756 non-null    object
 3   addr_gu    756 non-null    object
 4   addr_dong  756 non-null    object
dtypes: object(5)
memory usage: 35.4+ KB


In [35]:
# 유치원 형태 통일 (법인/사인, 단설/병설)
kin['kin_type'] = kin['kin_type'].str[:-4]
kin.head()

,kin_type,kin_name,kin_addr,addr_gu,addr_dong
0,사립,가람유치원,서울특별시 강동구 상암로63길 39,강동구,명일2동
1,사립,가명유치원,서울특별시 중구 청파로 447-1,중구,중림동
2,사립,강남유정유치원,서울특별시 강남구 광평로34길 17,강남구,수서동
4,사립,강남유치원,서울특별시 동작구 상도로41가길 13,동작구,상도1동
5,사립,강동유정유치원,서울특별시 강동구 상일로10길 41,강동구,상일2동


### 구별 비율

In [38]:
kin_gu = kin.groupby(['addr_gu', 'kin_type'], as_index=False)['addr_dong'].count()
print(kin_gu)

   addr_gu kin_type  addr_dong
0      강남구       공립         17
1      강남구       사립         18
2      강동구       공립         11
3      강동구       사립         21
4      강북구       공립          7
5      강북구       사립         10
6      강서구       공립         18
7      강서구       사립         31
8      관악구       공립         16
9      관악구       사립         15
10     광진구       공립          8
11     광진구       사립         17
12     구로구       공립         14
13     구로구       사립         17
14     금천구       공립          7
15     금천구       사립          9
16     노원구       공립         20
17     노원구       사립         39
18     도봉구       공립         11
19     도봉구       사립         17
20    동대문구       공립          9
21    동대문구       사립         18
22     동작구       공립          8
23     동작구       사립         18
24     마포구       공립         13
25     마포구       사립         14
26    서대문구       공립          5
27    서대문구       사립         16
28     서초구       공립         10
29     서초구       사립         13
30     성동구       공립         14
31     성

In [44]:
# 비율 계산 함수

df = kin_gu

def calculate_school_type_ratio(dataframe):
    result = {}
    for _, row in dataframe.iterrows():
        구 = row['addr_gu']
        학교형태 = row['kin_type']
        학교수 = row['addr_dong']
        
        if 구 not in result:
            result[구] = {'공립': 0, '사립': 0}
        
        result[구][학교형태] += 학교수
    
    # 계산된 비율을 저장
    for 구 in result:
        total = result[구]['공립'] + result[구]['사립']
        if total > 0:
            result[구]['공립'] = result[구]['공립'] * 100 / total
            result[구]['사립'] = result[구]['사립'] * 100 / total
        else:
            result[구]['공립'] = 0
            result[구]['사립'] = 0
    
    return result

result = calculate_school_type_ratio(df)
kin_gu_ratio = pd.DataFrame(result)

kin_gu_ratio = kin_gu_ratio.T
kin_gu_ratio = kin_gu_ratio.reset_index()
kin_gu_ratio = kin_gu_ratio.rename(columns={'index': '구'})
kin_gu_ratio = kin_gu_ratio[['구', '공립']]
kin_gu_ratio.head()

,구,공립
0,강남구,48.571429
1,강동구,34.375000
2,강북구,41.176471
3,강서구,36.734694
4,관악구,51.612903


In [47]:
# csv로 저장
kin_gu_ratio.to_csv("서울시 자치구별 국공립유치원 비율.csv")

### 동별 비율

In [46]:
kin_dong = kin.groupby(['addr_gu', 'addr_dong', 'kin_type'], as_index= False)['kin_name'].count()
print(kin_dong)

    addr_gu addr_dong kin_type  kin_name
0       강남구      개포2동       공립         1
1       강남구      개포2동       사립         1
2       강남구      개포3동       사립         2
3       강남구      개포4동       공립         1
4       강남구      개포4동       사립         1
..      ...       ...      ...       ...
492     중랑구      신내2동       공립         2
493     중랑구      신내2동       사립         3
494     중랑구      중화1동       공립         1
495     중랑구      중화1동       사립         2
496     중랑구      중화2동       사립         1

[497 rows x 4 columns]


In [48]:
# 비율 계산

df = kin_dong

def calculate_school_type_ratio(dataframe):
    result = {}
    for _, row in dataframe.iterrows():
        구 = row['addr_gu']
        동 = row['addr_dong']
        학교형태 = row['kin_type']
        학교수 = row['kin_name']
        
        if 동 not in result:
            result[동] = {'공립': 0, '사립': 0}
        
        result[동][학교형태] += 학교수
    
    # 계산된 비율을 저장
    for 동 in result:
        total = result[동]['공립'] + result[동]['사립']
        if total > 0:
            result[동]['공립'] = result[동]['공립'] * 100 / total
            result[동]['사립'] = result[동]['사립'] * 100 / total
        else:
            result[동]['공립'] = 0
            result[동]['사립'] = 0
    
    return result

result = calculate_school_type_ratio(df)
kin_dong_ratio = pd.DataFrame(result)

kin_dong_ratio = kin_dong_ratio.T
kin_dong_ratio = kin_dong_ratio.reset_index()
kin_dong_ratio = kin_dong_ratio.rename(columns={'index': '동'})
kin_dong_ratio = kin_dong_ratio[['동', '공립']]
kin_dong_ratio.head()

,동,공립
0,개포2동,50.0
1,개포3동,0.0
2,개포4동,50.0
3,논현1동,100.0
4,논현2동,50.0


In [49]:
# csv로 저장
kin_dong_ratio.to_csv("서울시 행정동별 국공립유치원 비율.csv")